In [1]:
# Parameters
DB_PATH = "C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-92\\test_notebooks_execute_element1\\interactions.db"
SEARCH_LEDGER_PATH = "C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-92\\test_notebooks_execute_element1\\search_telemetry.json"


# Elements Reporting

Generate a lightweight report from an Elements graph by simulating the run, capturing the execution trace, and writing the results to disk for downstream automation.

In [2]:
GRAPH_PRESET = globals().get('GRAPH_PRESET', 'insight_report')
GRAPH_FILE = globals().get('GRAPH_FILE', None)
RUN_OVERRIDES = globals().get('RUN_OVERRIDES', '{}')
REPORT_PATH = globals().get('REPORT_PATH', './_papermill/elements-report.md')

In [3]:
from pathlib import Path
from datetime import datetime
import json
import sys


def _discover_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'PROJECT_OVERVIEW.md').exists():
            return candidate
    return start


REPO_ROOT = _discover_repo_root(Path.cwd().resolve())
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from datalab.scripts import elements as elements_utils

REPORT_PATH = Path(REPORT_PATH)
REPORT_PATH.parent.mkdir(parents=True, exist_ok=True)
TRACE_PATH = REPORT_PATH.with_suffix('.trace.json')
(REPO_ROOT, REPORT_PATH, TRACE_PATH)

(WindowsPath('D:/Files/Code 3/ChatAI-DataLab'),
 WindowsPath('_papermill/elements-report.md'),
 WindowsPath('_papermill/elements-report.trace.json'))

In [4]:
graph_payload = elements_utils.load_graph(GRAPH_FILE, preset=GRAPH_PRESET) if GRAPH_FILE else elements_utils.load_graph(preset=GRAPH_PRESET)
graph_summary = elements_utils.graph_summary(graph_payload)
graph_summary

{'name': 'Insight Report',
 'tenantId': 'lab',
 'workspaceId': 'default',
 'node_count': 3,
 'edge_count': 2,
 'node_types': {'prompt': 1, 'llm': 1, 'notebook': 1},
 'tags': ['report']}

In [5]:
execution_plan = elements_utils.build_execution_plan(graph_payload)
execution_plan

[{'step': 1, 'node': 'node_report_prompt', 'type': 'prompt', 'depends_on': []},
 {'step': 2,
  'node': 'node_report_llm',
  'type': 'llm',
  'depends_on': ['node_report_prompt']},
 {'step': 3,
  'node': 'node_report_notebook',
  'type': 'notebook',
  'depends_on': ['node_report_llm']}]

In [6]:
def _parse_overrides(raw: str) -> dict[str, dict[str, object]]:
    raw = raw.strip() if raw else ''
    if not raw:
        return {}
    payload = json.loads(raw)
    if not isinstance(payload, dict):
        raise ValueError('Overrides must be a JSON object of nodeId -> {props: {...}}')
    return payload

overrides = _parse_overrides(RUN_OVERRIDES)
overrides

{}

In [7]:
run_result = elements_utils.simulate_graph(graph_payload, overrides=overrides)
elements_utils.export_trace(run_result['trace'], TRACE_PATH)
report_lines = [
    '# Elements Report',
    f"Generated: {datetime.utcnow().isoformat()}Z",
    f"Graph: {graph_summary['name']} ({graph_summary['node_count']} nodes)",
    f"Tags: {', '.join(graph_summary.get('tags', [])) or 'n/a'}",
    '',
    '## Outputs',
    json.dumps(run_result['outputs'], indent=2),
    '',
    '## Trace',
]
for entry in run_result['trace']:
    report_lines.append(
        f"- {entry['id']} ({entry['type']}): inputs={entry['inputs']} -> outputs={entry['outputs']}"
    )
REPORT_PATH.write_text('\n'.join(report_lines), encoding='utf-8')
{'report': str(REPORT_PATH), 'trace': str(TRACE_PATH), 'status': run_result['status']}

C:\Users\Nihil\AppData\Local\Temp\ipykernel_24812\2766136791.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f"Generated: {datetime.utcnow().isoformat()}Z",


{'report': '_papermill\\elements-report.md',
 'trace': '_papermill\\elements-report.trace.json',
 'status': 'succeeded'}